In [1]:
import string
from nltk.util import ngrams
from collections import OrderedDict, defaultdict
from datetime import datetime
from tqdm import tqdm

import numpy as np
from statistics import mean

In [14]:
class GoodTuringSmoothing:
    def __init__(self,ngramHigherOrder,ngramLowerOrder,K):
        self.k = K
        self.V = 0
        self.ngramHigherOrder = ngramHigherOrder
        self.ngramLowerOrder = ngramLowerOrder
        
        self.vocab_dict = defaultdict()
        self.ngram_dicts = []
        self.final_prob_ngram_dicts = []
        self.ngrams_nc_dicts = []
        self.data =['they do not', 'they do to', 'i hope so', 'okay you are gonna need to learn how to lie', 'like my fear of wearing pastels', "the 'real you'", 'what good stuff', 'i figured you would get to the good stuff eventually', 'me this endless blonde babble I am like boring myself', 'do you listen to this crap', 'you always been this selfish', 'then that is all you had to say', 'have fun tonight', 'i believe we share an art instructor', 'you know chastity', 'looks like things worked out tonight huh', 'she is not a', 'you have my word as a gentleman', 'you got something on your mind', 'well there is someone i think might be', 'that is because it is such a nice one', 'right see you are ready for the quiz', "c'esc ma tete this is my head", 'let me see what i can do', 'gosh if only we could find kat a boyfriend', 'that is a shame', 'seems like she could get a date easy enough', 'not the hacking and gagging and spitting part please', "i do not have to be home 'til two", 'i have to be home in twenty minutes', 'is he oily or dry', 'would you mind getting me a drink cameron', 'where did he go he was just here', 'you might wanna think about it', 'did you change your hair', 'you are concentrating awfully hard considering it is gym class', 'hey sweet cheeks', 'patrick is that a', 'it is just you', 'no i just wanted', 'i just wanted', 'you set me up', 'so did you', 'you looked beautiful last night you know', 'i guess I shall  never know will i', 'i guess i thought i was protecting you', 'I am not stupid enough to repeat your mistakes', 'why did not you tell me', 'you did what', 'now i do back then was a different story', 'but you hate joey', 'he was like a total babe', 'in 9th for a month', 'joey never told you we went out did he', 'like you care', 'can we go now', 'you are so completely unbalanced', 'it is shakespeare maybe you have heard of him', 'like I am supposed to know what that even means', 'what do you think', 'oh my god does this mean you are becoming normal', 'can you at least start wearing a bra', 'nowhere hi daddy', 'where have you been', 'I am missing something', "he is not a 'hot rod' whatever that is", 'the prom kat has a date', 'because she will scare them away', 'just for a minute', 'wear the belly before you go', 'it is just a party daddy', 'oh god it is starting', "if kat's not going you are not going", 'daddy people expect me to be there', 'otherwise known as an orgy', "and where're you going", 'exactly my point', 'but she does not want to date', 'but it is not fair she is a mutant daddy', 'what if she never starts dating', "just sent 'em through", 'always a pleasure brucie', 'no i m not', 'buttholus extremus but hey you are making progress', 'extremely unfortunate maneuver', 'you told me that part already', 'it is her favorite band', 'assail your ears for one night', 'okay i was not sure', 'dead at forty one', 'her favorite uncle', 'it is a lung cancer issue', 'number one she hates smokers', 'will bogey get bent', 'you got him involved', 'i thought you hated those people', 'that is what i just said', 'did she actually say she would go out with you', 'they always let felons sit in on honors biology', 'he seems like he thrives on danger', 'what makes you think he will do it', 'you wanna go out with him', 'what about him', 'the mewling rampalian wretch herself', "that is her bianca's sister", 'yeah just a minor encounter with the shrew', 'you could consecrate with her my friend', 'guess who just signed up for a tutor', "sure do my mom's from canada", 'you know french', 'he always have that shit eating grin', 'bianca stratford sophomore do not even think about it', 'who is she', 'you burn you pine you perish', 'that girl i', 'that I am used to', 'couple thousand most of them evil', 'how many people go here', 'how many people were in your old school', 'yeah a couple we are outnumbered by the cows though', 'i was kidding people actually live there', 'north actually how did you', 'so which dakota you from', "c'mon I am supposed to give you the tour", 'so they tell me', 'you the new guy', 'you get the girl', 'what is the worst', 'she kissed me', 'you makin any headway', "what'd you do to her", 'then go get her', 'cameron do you like the girl', 'she is partial to joey not me', "what 're you talking about", 'it is off the whole thing', 'cameron I am a little busy', 'do not make me do it man', 'gigglepuss is playing there tomorrow night', 'what have you got for me', 'yeah we will see', 'and why would i do that', 'leave my sister alone', 'your sister here', 'where ya goin', 'not at all', 'hey do you mind', 'they are running the rest of me next month', 'enough with the barbie n ken shit i know', 'get her to act like a human', 'how did you do it', "a deal's a deal", 'it is about time', 'forget her sister then', 'a hundred bucks a date', 'i just upped my price', 'I am on it', 'when i shell out fifty i expect results', 'fifty and you have got your man', 'take it or leave it this is not a negotiation', 'you are gonna pay me to take out some girl', 'you need money to take a girl out', 'you just said', 'sure sparky I shall  get right on it', 'two legs nice rack', 'what do you think', 'hey it is all for the higher good right', 'you better not fuck this up I am heavily invested', 'what we took bathes together when we were kids', 'you and verona', 'uh yeah we are old friend', 'i hear you are helpin verona', 'does this conversation have a purpose', 'we do not chat', 'nope just came by to chat', 'are you lost', 'william he asked me to meet him here', 'have you seen him', 'oh good something new and different for us', 'i did bianca a favor and it backfired', 'i did not have a choice', "what'd he say", "you think this'll work", 'does it matter', 'neither has his heterosexuality', 'that is never been proven', 'william did not even go to high school', 'william would never have gone to a state school', 'the people at this school are so incredibly foul', 'but imagine the things he would say during sex', 'an attempted slit', 'just a little', 'he always look so', 'patrick verona random skid', 'don t you even dare', 'oh bianca can you get me my freshman yearbook', 'because i like to torture you', 'is that right', 'a fender strat you bought this', 'i did not care about the money', 'it was not like that', 'it gets worse you still have your freshman yearbook', 'that s completely adorable', "my grandmother's", 'it is just something i had you know', 'how did you get a tux at the last minute', 'answer the question patrick', 'you need therapy has anyone ever told you that', 'you tell me', 'create a little drama start a new rumor what', 'because i do not want to it is a stupid tradition', 'no i will not go with you', 'you know what i mean', 'is that a request or a command', 'go to the prom with me', 'no one else knows', 'you are sweet and sexy and completely hot for me', 'no something real something no one else knows', 'i hate peas', 'tell me something true', "i know the porn career's a lie", 'for', 'you up for it', 'you never disappointed me', 'then you screwed up', 'something like that', 'acting the way we do', 'so what is your excuse', 'yeah well do not let it get out', 'a soft side who knew', 'i dazzled him with my wit', 'the partridge family', "forget it I am stayin'", 'put your right foot there', 'try lookin at it from this angle', "c'mon it is not that bad", 'look up sunshine', 'other than my upchuck reflex nothing', 'so what did i have an effect on', "you 're so", 'i heard there was a poetry reading', 'what are you doing here', 'he just wants me to be someone I am not', 'I am gettin there', 'oh so now you think you know me', 'my father would not approve of that that', 'start a band', 'i should do this', "why'd you lie", "then why'd you ask", 'no you were not', 'were you in jail', 'when you were gone last year where were you', 'and I am in control of it', 'kat wake up', 'you know what they say', 'i thought you were above all that', 'i hate him', "why'd you let him get to you", 'just let me sit down', 'like you could find one', 'sure i do', 'you do not care if i die', 'i told you', "why 're you doing this", 'this is so patronizing', 'i know just let me sleep', 'i just need to lie down for awhile', 'you are not okay', 'I am fine I am', 'funny you are the only one', 'i say do what you wanna do', 'why do not you', 'you know who the raincoats are', 'that is what you want is not it', 'hey great show huh', 'you are not a big talker are you', 'are you following me', 'you covered in my vomit', 'and why would i do that', 'you hate me do not you', 'i know a lot more than that', 'oh right friday', 'pick you up friday then', 'my mission in life', "there is a way to get a guy's attention", 'sweating like a pig actually and yourself', "i mean wo man how ya doin'", 'when i go', 'bianca did what', 'parts of it', 'so tell me about this dance was it fun', 'was that your sister', 'so what i want does not matter', 'as a parent that is my right', 'because you are making decisions for me', 'why cannot we agree on this', 'i thought you were punishing me', 'is this about sarah lawrence you punishing me', 'then tell them i had a seizure', 'my insurance does not cover pms', 'just smack her now', 'the point is kat people perceive you as somewhat', 'expressing my opinion is not a terrorist action', 'kat a fan too', 'yeah i guess', 'cool pictures you a fan', 'hey there tired of breathing', 'sweet love renew thy force', 'you were right she is still pissed', 'so you got cozy with she who stings', 'I am likin you guys better', 'ever been to club skunk', 'just for now', "are you telling me I am a 'non smoker'", 'we are your guys', 'so dorsey can get the girl', 'that is where we can help you with kat', 'see you next week', 'you are completely demented', 'what is to discuss', 'touch of the flu', 'with the teeth of your zipper', 'it was a bratwurst i was eating lunch', 'i missed you', 'that s what i thought', 'did i have anything to say about it', 'your daughters went to the prom', \
                   'what just happened', 'but who what', 'have a great time honey', 'pirate no question', 'you are not helping', 'what is a synonym for throbbing', 'make anyone cry today', 'in the microwave', 'we should have seen land', 'we will all go crazy', "he is the devil's child", 'ah leave him alone he is doing no harm', 'what are you listening to chicken ass', "the water's going putrid in the barrels", 'if god wills it', 'blind faith is what i consider heresy', 'do not you realize your words could be considered heretical', 'is that all that interests you gold', 'if they agree to follow me yes', 'your life and that of others', 'excellency you are right', 'i am your eminence', 'you say asia can be found by sailing west', 'what a tragedy what a waste of a life', 'you can see for yourself', 'the judgment is ours', 'what is it now tell me', 'i am busy inside', 'cannot you stay with us a little', 'they took everything', 'they tried but i did not let them', 'you do not usually ask', 'beatrix i want to ask you something', 'that is something you cannot decide', 'i do not want you to wait for me', 'I am not asking you to swear to anything', 'she said yes', 'i find that hard to believe', 'are we going to argue', 'i have not given you much of a life', 'i could be gone for years', 'i am afraid this is not the worst news', 'how far from here', 'viceroy of the west indies', 'appointment to what', 'my letters of appointment', 'yes i remember', 'don alonso de bobadilla', 'bartolome and giacomo colon', 'may i ask by whom', 'he never had one except aboard my ships', 'what does he say', 'what are you listening to', 'there must be a passage to that other ocean', 'how are you feeling fernando', 'this time with me', 'i have to explore the mainland', 'yes yes i do on all of them', 'you promise do you swear on st christopher', "there'll be a time", 'i want to go with you', 'all of them created by people like me', 'roofs towers palaces spires', 'what do you see', 'look out of that window', 'all i have to do is call the guards', 'what do i have so many already', 'for a commoner', 'you defend yourself admirably', 'forgive me don colon but what about gold', 'then you are too ambitious', 'I am not bargaining', 'you never learned how to speak my language', 'you did the same to your god', 'ask him if he will help', 'he has medicine tell him we admire his people', 'and also to bring medicine', 'chief says he has a god', 'to bring the word of god', 'chief says how many', 'tell the chief we thank him', 'you come you speak first', 'what is it a tribe an island', 'say not here cuba', 'oh so you are a new man', 'new worlds create new people', 'i knew you would', 'i have to disagree', 'you will always be older than me father', 'i suppose we are both old men now', 'give me absolution', 'i believed in you', 'you are bound by an oath father', 'what are you saying', 'i am listening my son', 'forgive me father for i have sinned', 'in nomine patris et filius et spiritus sancti', 'all of them just lies', 'you must not give way to despair you must wait', 'I have been contradicted all my life eternity', 'passion is something one cannot control', 'I shall  try to remember that marchena', 'so was christ', 'esdras is a jew', 'the calculations of toscanelli marin de tyr esdras', 'how can you be so certain', 'why do you wish to sail west', 'how did you manage it', 'that is what it says', 'god that is in a week', 'do they have such thoughts', 'thirty seven your majesty and you', 'how old are you senor colon', 'you show no inclination to speak otherwise', 'may i speak freely', 'surely you can do anything you want', 'i cannot ignore the verdict of my council', 'that she was impregnable', 'what did they say about granada before today', 'the ocean is uncrossable', 'you will regret this', 'savagery is what monkeys understand', 'don moxica we all have to work', 'my horse does not work', 'we cannot raise the wheel without it', 'we do not need to know', 'the indians have no such word don moxica', 'we cannot be', 'what is he doing', 'what do you read', 'come over here', 'and what do you think mendez', 'god be with us admiral', 'you are right let the men decide', 'you tell that to them', 'jesus maria i should have never listened to you', 'the land is there i know it', 'you think i do not know that', 'we are on the verge of a mutiny colon', 'you must be mad', 'six days ago yes', 'you lied you cheated we are way past 750 leagues', 'where can i meet this man', 'because he is not afraid of me', 'i know i should not tolerate his impertinence', 'he must be replaced', 'then what do you suggest don sanchez', 'is this true brother buyl', 'into a monk', 'never your majesty although', 'yes your majesty', 'is that the man i knew treasurer sanchez', 'officers there is your killer do your duty arrest him', 'i do not think it is abuse i think it is torture', 'I am abused do not you think', 'can i see your back', 'your father put cigarettes out on you', "your father blamed you for your mother's blindness", 'back in the czech republic', 'your mother was blind', 'give me another one please', 'what else do you need', 'no no way', 'i would say half half is fair', 'what is your cut how much', 'what about my movie rights book rights', 'do not worry about him think about yourself', 'no he is here shit', 'what about oleg', 'i was all of these', 'delusions and paranoia', 'you bring the cigarettes']
    def _find_best_fit_slope(self,x,y):
        m = (( mean(x)*mean(y) - mean(x*y) ) / 
              ( mean(x)** 2 - mean(x**2)))

        return m

    def _find_best_fit_intercept(self,x,y,m):
        c = mean(y) - m*mean(x)
        return c

    def _find_good_turing_adjustCount(self,c, k, nc_dict):

        adjust_count = ( ( (( c + 1)*( nc_dict[c + 1] / nc_dict[c])) - ( c * (k+1) * nc_dict[k+1] / nc_dict[1]) ) /
                         ( 1 - (( k + 1)*nc_dict[k + 1] / nc_dict[1]) )
                       )
        return adjust_count
    def _loadCorpus(self,n):
        ngram_dict = defaultdict(int)
        lines = self.data
        
        lines = ['<start> ' + x + ' <end>' for x in lines]
        for line in lines:
            temp_l = line.split()
            # print(temp_l)
            i = 0
            j = 0
            for word in temp_l :
                j = 0
                for l in word :
                    if l in '!"#$%&\'()*+,-./:;=?@[\\]^_`{|}~':
                        if l == "'":
                            if j+1<len(word) and word[j+1] == 's':
                                j = j + 1
                                continue
                        word = word.replace(l," ")
                        #print(j,word[j])
                    j += 1
                temp_l[i] = word.lower()
                i=i+1 
            content = " ".join(temp_l)
            token = content.split()
            if not token:
                continue
            ngrams_list = list(ngrams(token, n))
            # Building Vocabulary
            for word in token:
                if word not in self.vocab_dict:
                    self.vocab_dict[word] = 1
                else:
                    self.vocab_dict[word] += 1
            # Building Ngram
            if ngrams_list == []:
                pass
            else:
                for t in ngrams_list:
                    sen = ' '.join(t)
                    ngram_dict[sen] += 1
        return ngram_dict
    
    def _find_frequency_of_frequency_count(self,ngram_dict,k, n, V, token_len):
        nc_dict = {}
        nc_dict[0] = V**n - token_len

        for key in ngram_dict:
            if ngram_dict[key] <= k + 1:
                if ngram_dict[key] not in nc_dict:
                    nc_dict[ ngram_dict[key]] = 1
                else:
                    nc_dict[ ngram_dict[key] ] += 1
          
        val_present = True
        for i in range(1,7):
            if i not in nc_dict:
                val_present = False
                break
        if val_present == True:
            return nc_dict
        data_pts = {}
        i = 0
        for key in ngram_dict:
            if ngram_dict[key] not in data_pts:
                    data_pts[ ngram_dict[key] ] = 1
                    i += 1
            if i >5:
                break
                
        for key in ngram_dict:
            if ngram_dict[key] in data_pts:
                data_pts[ ngram_dict[key] ] += 1

        x_coor = [ np.log(item) for item in data_pts ]
        y_coor = [ np.log( data_pts[item] ) for item in data_pts ]
        x = np.array(x_coor, dtype = np.float64)
        y = np.array(y_coor , dtype = np.float64)
    
        slope_m = self._find_best_fit_slope(x,y)
        intercept_c = self._find_best_fit_intercept(x,y,slope_m)
    
        for i in range(1,(k+2)):
            if i not in nc_dict:
                nc_dict[i] = (slope_m*i) + intercept_c
        
        return nc_dict

    
    def _calculate_ngram_prob(self):

        for i in range(self.ngramHigherOrder,self.ngramLowerOrder,-1):
            n_gram_prob_dict = OrderedDict()
            gram_dict = self.ngram_dicts[i-2-1]
            one_less_gram_dict = self.ngram_dicts[i-2-2]
            nc_count = self.ngrams_nc_dicts[i-2-1]
            
            for i,sentence in enumerate(gram_dict):
    
                sent_token = sentence.split()
    
                one_less_sentnence = " ".join(sent_token[:-1])
    
                sen_count = gram_dict[sentence]
                one_less_count = one_less_gram_dict[one_less_sentnence]
    
                if gram_dict[sentence] <= self.k or (sentence not in gram_dict):
                    sen_count = self._find_good_turing_adjustCount(gram_dict[sentence],self.k,nc_count)
    
                if one_less_gram_dict[one_less_sentnence] <=self.k or (one_less_sentnence not in one_less_gram_dict):
                    one_less_count = self._find_good_turing_adjustCount(one_less_gram_dict[one_less_sentnence],self.k,nc_count)
    
                prob = sen_count/one_less_count
                try:
                    if one_less_sentnence not in n_gram_prob_dict:
                        n_gram_prob_dict[one_less_sentnence]=[]
                        n_gram_prob_dict[one_less_sentnence].append([prob,sent_token[-1]])
                    else :
                        n_gram_prob_dict[one_less_sentnence].append([prob,sent_token[-1]])
                except Exception as e:
                    pass
            prob = None
            sent_token = None
            one_less_sentnence = None
            self.final_prob_ngram_dicts.append(n_gram_prob_dict)

    def _sort_prob_word_dict_good_turing(self):
    
        final_ngram_prob_dict = [x for x in reversed(self.final_prob_ngram_dicts)]
        new_list =[]
        
        for i in range(self.ngramLowerOrder-self.ngramLowerOrder,self.ngramHigherOrder-self.ngramLowerOrder):
            ngram_dict = final_ngram_prob_dict[i]
            for key in ngram_dict:
                if len(ngram_dict[key])>1:
                    ngram_dict[key] = sorted(ngram_dict[key],reverse=True)[:4]
            new_list.append(ngram_dict)
        self.final_prob_ngram_dicts = new_list
    
        
    def MakeModel(self):
        for i in range(self.ngramHigherOrder,self.ngramLowerOrder,-1):
            self.ngram_dicts.append(self._loadCorpus(i))

        self.V = len(self.vocab_dict)

        for i in range(self.ngramLowerOrder,self.ngramHigherOrder):
            self.ngrams_nc_dicts.append(self._find_frequency_of_frequency_count(self.ngram_dicts[i-2], self.k, i, self.V, len(self.ngram_dicts[i-2])))

        self._calculate_ngram_prob()
        self._sort_prob_word_dict_good_turing()

    
    def _clean_sentence(self,sen):
        """
        Funtion to remove punctuations from the given input sentence and covert them to lowercase.
        arg: string
        returns: string
        """
        temp_l = sen.split()
        i = 0
        j = 0
        
        for word in temp_l :
            j = 0
            for l in word :
                if l in string.punctuation:
                    if l == "'":
                        if j+1<len(word) and word[j+1] == 's':
                            j = j + 1
                            continue
                    word = word.replace(l," ")
                j += 1
    
            temp_l[i] = word.lower()
            i=i+1   
        content = " ".join(temp_l)
        return content

    def _predict_sentence(self,input_sen):
        token = input_sen.split()
        pred = []
        
        for i in range(self.ngramHigherOrder-self.ngramLowerOrder,self.ngramLowerOrder-self.ngramLowerOrder,-1):
            ngram_dict = self.final_prob_ngram_dicts[i-2]
            
            n = len(list(ngram_dict.keys())[0].split())
           
            if " ".join(token[-n:]) in ngram_dict and ngram_dict[" ".join(token[-n:])][0][0]>0:
                return ngram_dict[" ".join(token[-n:])][0]
        return []

    def Predict(self):
        while True:
            inp = input("Enter the sentence to predict : ")
            if inp== "exit": 
                break
            if inp.split() == [] and len(inp.split())>0:
                break
            i=0
            inp = self._clean_sentence(inp)
            inp =  "<start> "+inp
            while i<2:
                w = self._predict_sentence(inp)
                if w !=[] and w[0]>0.1 :
                    i+=1
                    inp = inp + " "+w[1]
                else:
                    break
            print(inp.replace(" <end>",".").replace("<start> ",""))

In [15]:
g = GoodTuringSmoothing(7,2,2)
g.MakeModel()
g.Predict()

Enter the sentence to predict :  are you


are you telling me


Enter the sentence to predict :  exit
